In [0]:
%run ../../_utils

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql.functions import when, to_date, col, dayofweek, date_format, month, year
from pyspark.sql.types import StringType, BooleanType


# Camada Silver

Na camada silver, limpezas e ajustes em dados devem ser aplicados

Caso seja possível, enriquecer os dados e extrair dados também deve acontecer nessa camada (minha definição)

In [0]:
tb_name = "silver.olist_orders"
dataset_location = "olist_orders_dataset"
target_location = f"dbfs:/FileStore/delta/silver/brazilian_ecommerce/{dataset_location}"

## 1 - Data ingestion

In [0]:
df = spark.read.table("bronze.olist_orders") # leituira da delta table

In [0]:
df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [0]:
display(df.take(10))

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02T10:56:33Z,2017-10-02T11:07:15Z,2017-10-04T19:55:00Z,2017-10-10T21:25:13Z,2017-10-18T00:00:00Z
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24T20:41:37Z,2018-07-26T03:24:27Z,2018-07-26T14:31:00Z,2018-08-07T15:27:45Z,2018-08-13T00:00:00Z
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08T08:38:49Z,2018-08-08T08:55:23Z,2018-08-08T13:50:00Z,2018-08-17T18:06:29Z,2018-09-04T00:00:00Z
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18T19:28:06Z,2017-11-18T19:45:59Z,2017-11-22T13:39:59Z,2017-12-02T00:28:42Z,2017-12-15T00:00:00Z
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13T21:18:39Z,2018-02-13T22:20:29Z,2018-02-14T19:46:34Z,2018-02-16T18:17:02Z,2018-02-26T00:00:00Z
a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09T21:57:05Z,2017-07-09T22:10:13Z,2017-07-11T14:58:04Z,2017-07-26T10:57:55Z,2017-08-01T00:00:00Z
136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11T12:22:08Z,2017-04-13T13:25:17Z,null,null,2017-05-09T00:00:00Z
6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16T13:10:30Z,2017-05-16T13:22:11Z,2017-05-22T10:07:46Z,2017-05-26T12:55:51Z,2017-06-07T00:00:00Z
76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,2017-01-23T18:29:09Z,2017-01-25T02:50:47Z,2017-01-26T14:16:31Z,2017-02-02T14:08:10Z,2017-03-06T00:00:00Z
e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,2017-07-29T11:55:02Z,2017-07-29T12:05:32Z,2017-08-10T19:45:24Z,2017-08-16T17:14:30Z,2017-08-23T00:00:00Z



## 2 - Data Munging

Processo de limpeza e normalizações necessárias

Como esses dados já são bem tratados, vamos apenas atentar para extração ou "enriquecimento" de dados.

Inicialmente iremos carregar dados para agregações:
 - Campo mes/ano para calcular vendas mensais, trimestrais e etc

Podemos nos focar no tempo decorrido de cada etapa, por exemplo:
 - tempo até a aprovação (em minutos ou segundos)
 - tempo de entrega (em dias)
 - tempo total da compra até a entrega (em dias)
 - atraso (divergencia entre tempo estimado e o entregue)

 Além disso, podemos trazer dados que auxiliem na analise do padrão de compra por data
  - dia da semana
  - é fim de semana?

aqui poderia estressar e ir até para coisas do tipo, mas daí nao daria tempo no projeto:
pandas_market_calendars
 - é feriado?
 - qual feriado
 - dias até o próximo feriado - para entender padrões de compra próximo a feriados

### month_year

mes/ano da venda

Campos desse tipo ajudam em calculos vendas mensais

In [0]:
df = df.withColumns(
    {
        "month_year": date_format("order_purchase_timestamp", "MM-yyy"),
        "order_purchase_month": month("order_purchase_timestamp"),
        "order_purchase_year": year("order_purchase_timestamp"),
    }
)


### minutes_to_approve

Tempo decorrido até a aprovação do pedido

time delta entre ``order_purchase_timestamp`` e ``order_approved_at``

In [0]:
df = get_diff_between_dates(
    df,
    col_1="order_approved_at",
    col_2="order_purchase_timestamp",
    delta_type="minutes",
    col_name="minutes_to_approve",
)


### days_to_deliver
#### hours e days

Tempo total da transportadora até chegar ao destinatário

time delta entre ``order_delivered_carrier_date`` e ``order_delivered_customer_date``

In [0]:
df = get_diff_between_dates(
    df,
    col_1="order_delivered_customer_date",
    col_2="order_delivered_carrier_date",
    delta_type="days",
    col_name="days_to_deliver",
)
df = get_diff_between_dates(
    df,
    col_1="order_delivered_customer_date",
    col_2="order_delivered_carrier_date",
    delta_type="hours",
    col_name="hours_to_deliver",
)


### total_elapsed_time

Tempo total decorrido em que o cliente solicitou a compra e recebeu em sua casa

In [0]:
# extrair o tempo até a entrega (em dias)
df = get_diff_between_dates(
    df,
    col_1="order_delivered_customer_date",
    col_2="order_purchase_timestamp",
    delta_type="days",
    col_name="total_elapsed_days",
)
# extrair o tempo até a entrega (em horas)
df = get_diff_between_dates(
    df,
    col_1="order_delivered_customer_date",
    col_2="order_purchase_timestamp",
    delta_type="hours",
    col_name="total_elapsed_hours",
)


### delay_time and overdue?

``overdue``>: True or false - quando o tempo estimado for MENOR q o tempo da entrega, significa q é overdue (atraso)

aqui como é uma base estática, não faz sentido adicionar o atrasado com sentido de "entrega está atrasada?", apenas para o sentido de se a "entrega ocorreu com atraso". 
E por isso talvez fizesse sentido adicionar teste no "order_status" para garantir comportamento ideal.[]


``delay_time``>: estimado x real



In [0]:
df = df.withColumn(
    "overdue",
    when(
        col("order_estimated_delivery_date") < col("order_delivered_customer_date"),
        True,
    ).otherwise(False),
)

# pegar os atrasados e informar o tempo de atraso (real - previsto)
df = df.withColumn(
    "delay_time",
    when(
        col("overdue"),
        col("order_delivered_customer_date").cast("long")
        - col("order_estimated_delivery_date").cast("long"),
    ).otherwise(0),
)

# converter para dias e para horas
df = df.withColumn("delay_hours", round(col("delay_time") / 3600, 2))
df = df.withColumn("delay_days", round(col("delay_time") / (24 * 3600), 2)).drop(
    "delay_time"
)

### day_of_week

In [0]:
df = df.withColumn("number_day_of_week", dayofweek(to_date(col("order_purchase_timestamp"))))

# Traduzir os números do dia da semana para o português
df = df.withColumn(
    "day_of_week",
    when(df["number_day_of_week"] == 1, "Domingo")
    .when(df["number_day_of_week"] == 2, "Segunda-feira")
    .when(df["number_day_of_week"] == 3, "Terça-feira")
    .when(df["number_day_of_week"] == 4, "Quarta-feira")
    .when(df["number_day_of_week"] == 5, "Quinta-feira")
    .when(df["number_day_of_week"] == 6, "Sexta-feira")
    .when(df["number_day_of_week"] == 7, "Sábado"),
)


### weekend?

criar um campo para informar se a data de compra é em um fim de semana (ajuda também para verificar a logística)

In [0]:
df = df.withColumn(
    "weekend",
    when(col("day_of_week").isin(["Sábado", "Domingo"]), True).otherwise(False),
)

In [0]:
display(df.take(10))

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,month_year,order_purchase_month,order_purchase_year,minutes_to_approve,days_to_deliver,hours_to_deliver,total_elapsed_days,total_elapsed_hours,overdue,delay_hours,delay_days,number_day_of_week,day_of_week,weekend
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02T10:56:33Z,2017-10-02T11:07:15Z,2017-10-04T19:55:00Z,2017-10-10T21:25:13Z,2017-10-18T00:00:00Z,10-2017,10,2017,10.7,6.06,145.5,8.44,202.48,false,0.0,0.0,2,Segunda-feira,false
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24T20:41:37Z,2018-07-26T03:24:27Z,2018-07-26T14:31:00Z,2018-08-07T15:27:45Z,2018-08-13T00:00:00Z,07-2018,7,2018,1842.83,12.04,288.95,13.78,330.77,false,0.0,0.0,3,Terça-feira,false
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08T08:38:49Z,2018-08-08T08:55:23Z,2018-08-08T13:50:00Z,2018-08-17T18:06:29Z,2018-09-04T00:00:00Z,08-2018,8,2018,16.57,9.18,220.27,9.39,225.46,false,0.0,0.0,4,Quarta-feira,false
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18T19:28:06Z,2017-11-18T19:45:59Z,2017-11-22T13:39:59Z,2017-12-02T00:28:42Z,2017-12-15T00:00:00Z,11-2017,11,2017,17.88,9.45,226.81,13.21,317.01,false,0.0,0.0,7,Sábado,true
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13T21:18:39Z,2018-02-13T22:20:29Z,2018-02-14T19:46:34Z,2018-02-16T18:17:02Z,2018-02-26T00:00:00Z,02-2018,2,2018,61.83,1.94,46.51,2.87,68.97,false,0.0,0.0,3,Terça-feira,false
a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09T21:57:05Z,2017-07-09T22:10:13Z,2017-07-11T14:58:04Z,2017-07-26T10:57:55Z,2017-08-01T00:00:00Z,07-2017,7,2017,13.13,14.83,356.0,16.54,397.01,false,0.0,0.0,1,Domingo,true
136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11T12:22:08Z,2017-04-13T13:25:17Z,null,null,2017-05-09T00:00:00Z,04-2017,4,2017,2943.15,null,null,null,null,false,0.0,0.0,3,Terça-feira,false
6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16T13:10:30Z,2017-05-16T13:22:11Z,2017-05-22T10:07:46Z,2017-05-26T12:55:51Z,2017-06-07T00:00:00Z,05-2017,5,2017,11.68,4.12,98.8,9.99,239.76,false,0.0,0.0,3,Terça-feira,false
76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,2017-01-23T18:29:09Z,2017-01-25T02:50:47Z,2017-01-26T14:16:31Z,2017-02-02T14:08:10Z,2017-03-06T00:00:00Z,01-2017,1,2017,1941.63,6.99,167.86,9.82,235.65,false,0.0,0.0,2,Segunda-feira,false
e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,2017-07-29T11:55:02Z,2017-07-29T12:05:32Z,2017-08-10T19:45:24Z,2017-08-16T17:14:30Z,2017-08-23T00:00:00Z,07-2017,7,2017,10.5,5.9,141.49,18.22,437.32,false,0.0,0.0,7,Sábado,true



belezura!

Limpamos um cadim (nada), enriquecemos e transformamos algumas coisas e temos nosso dataset preparado para analises (até preditiva e prescritiva -> mas aí tem que fazer um pouco mais de cleanings e etc)


## Saving data

In [0]:
save_dataframe(df, format_mode="delta", table_name=tb_name, target_location=target_location)

[LOG] Saving silver.olist_orders delta on dbfs:/FileStore/delta/silver/brazilian_ecommerce/olist_orders_dataset... OK!



## create delta table

TODO: implementar UPSERT

o upsert serve para não precisar reescrever todos os dados, mas aproveitar do Delta para fazer um MERGE, caso um registro antigo tenha uma nova versão e INSERT para os dados que são novos

In [0]:
create_table(table_name=tb_name, target_location=target_location)

[LOG] Creating delta table silver.olist_orders on dbfs:/FileStore/delta/silver/brazilian_ecommerce/olist_orders_dataset... OK!


In [0]:
# exit para fechar a execução
dbutils.notebook.exit("OK")

In [0]:
%sql

select * from silver.olist_orders limit 10

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,month_year,order_purchase_month,order_purchase_year,minutes_to_approve,days_to_deliver,hours_to_deliver,total_elapsed_days,total_elapsed_hours,overdue,delay_hours,delay_days,number_day_of_week,day_of_week,weekend
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02T10:56:33Z,2017-10-02T11:07:15Z,2017-10-04T19:55:00Z,2017-10-10T21:25:13Z,2017-10-18T00:00:00Z,10-2017,10,2017,10.7,6.06,145.5,8.44,202.48,false,0.0,0.0,2,Segunda-feira,false
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24T20:41:37Z,2018-07-26T03:24:27Z,2018-07-26T14:31:00Z,2018-08-07T15:27:45Z,2018-08-13T00:00:00Z,07-2018,7,2018,1842.83,12.04,288.95,13.78,330.77,false,0.0,0.0,3,Terça-feira,false
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08T08:38:49Z,2018-08-08T08:55:23Z,2018-08-08T13:50:00Z,2018-08-17T18:06:29Z,2018-09-04T00:00:00Z,08-2018,8,2018,16.57,9.18,220.27,9.39,225.46,false,0.0,0.0,4,Quarta-feira,false
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18T19:28:06Z,2017-11-18T19:45:59Z,2017-11-22T13:39:59Z,2017-12-02T00:28:42Z,2017-12-15T00:00:00Z,11-2017,11,2017,17.88,9.45,226.81,13.21,317.01,false,0.0,0.0,7,Sábado,true
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13T21:18:39Z,2018-02-13T22:20:29Z,2018-02-14T19:46:34Z,2018-02-16T18:17:02Z,2018-02-26T00:00:00Z,02-2018,2,2018,61.83,1.94,46.51,2.87,68.97,false,0.0,0.0,3,Terça-feira,false
a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09T21:57:05Z,2017-07-09T22:10:13Z,2017-07-11T14:58:04Z,2017-07-26T10:57:55Z,2017-08-01T00:00:00Z,07-2017,7,2017,13.13,14.83,356.0,16.54,397.01,false,0.0,0.0,1,Domingo,true
136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11T12:22:08Z,2017-04-13T13:25:17Z,null,null,2017-05-09T00:00:00Z,04-2017,4,2017,2943.15,null,null,null,null,false,0.0,0.0,3,Terça-feira,false
6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16T13:10:30Z,2017-05-16T13:22:11Z,2017-05-22T10:07:46Z,2017-05-26T12:55:51Z,2017-06-07T00:00:00Z,05-2017,5,2017,11.68,4.12,98.8,9.99,239.76,false,0.0,0.0,3,Terça-feira,false
76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,2017-01-23T18:29:09Z,2017-01-25T02:50:47Z,2017-01-26T14:16:31Z,2017-02-02T14:08:10Z,2017-03-06T00:00:00Z,01-2017,1,2017,1941.63,6.99,167.86,9.82,235.65,false,0.0,0.0,2,Segunda-feira,false
e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,2017-07-29T11:55:02Z,2017-07-29T12:05:32Z,2017-08-10T19:45:24Z,2017-08-16T17:14:30Z,2017-08-23T00:00:00Z,07-2017,7,2017,10.5,5.9,141.49,18.22,437.32,false,0.0,0.0,7,Sábado,true
